In [2]:
import folium
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()



Successfully saved authorization token.


In [3]:
dataset = ee.ImageCollection('MODIS/061/MOD13A2')

# Initial date of interest (inclusive).
i_date = '2019-07-01'

# Final date of interest (exclusive).
f_date = '2020-07-01'

# Selection of appropriate bands and dates for LST.

# Select the NDVI band
ndvi = dataset.select('NDVI').filterDate(i_date, f_date)

# Define visualization parameters
ndviVis = {
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ]
}

In [4]:
lat = 33.7488
lon = -84.3877

poi = ee.Geometry.Point(lon, lat)

# Define a region of interest with a buffer zone of 100 km around point.
roi = poi.buffer(1e5)

In [5]:
ndvi = ndvi.filterBounds(roi)

In [6]:
ndvi

In [7]:
ndvi_img = ndvi.mean()
ndvi_img

In [8]:
my_map = folium.Map(location=[lat, lon], zoom_start = 10)

In [9]:
my_map

In [10]:
def add_ee_layer(my_map, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(my_map)
    my_map.add_child(folium.LayerControl())

In [11]:
add_ee_layer(my_map, ndvi_img, ndviVis, 'NDVI')

In [12]:
my_map

In [101]:
# Save to GeoTIFF
task = ee.batch.Export.image.toDrive(image=ndvi_img,
                                     description='ndvi_atlanta_'+str(i_date) + str(f_date),
                                     scale=30,
                                     region=roi,
                                     fileNamePrefix='ndvi_atlanta_'+str(i_date) + str(f_date),
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')

In [102]:
task.start()

In [103]:
task.status()

{'state': 'READY',
 'description': 'ndvi_atlanta_2019-07-012020-07-01',
 'creation_timestamp_ms': 1683045135586,
 'update_timestamp_ms': 1683045135586,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'SPXINPHPTMLEGZGVDL7TRQJQ',
 'name': 'projects/earthengine-legacy/operations/SPXINPHPTMLEGZGVDL7TRQJQ'}